In [1]:
from pypylon import pylon
import os
import yaml
import time

In [2]:
# !jt -t solarizedd -T -N -kl

In [3]:
from devices import Basler

PySpin not found, cant acquire from FLIR cameras


In [4]:
name='test'
start_t = time.perf_counter()
experiment = '%s_%s' %(name, time.strftime('%y%m%d_%H%M%S', time.localtime()))

In [5]:
config_path = '/home/julianarhee/Repositories/PIMAQ/config.yaml'
with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [6]:
tuples=[]
for camname, cam in config['cams'].items():
    tup = (config, camname, cam, experiment, start_t)
    tuples.append(tup)

In [7]:
camname, cam['type']

('right_top', 'Basler')

In [8]:
22297042 

22297042

In [9]:
info = pylon.DeviceInfo()
info.SetSerialNumber("22297042")
cam  = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice(info))

In [12]:
cam.Open()

In [14]:
cam.Close()

In [9]:
device = Basler(serial=cam['serial'], 
    start_t=start_t, 
    options=cam['options'],
    save=True,
    savedir=config['savedir'],
    experiment=experiment,
    name=camname,
    movie_format='ffmpeg',
    metadata_format='hdf5', 
    uncompressed=False, # setting to False always because you don't need to calibrate it
    preview=True,
    verbose=True, #args.verbose,
    strobe=cam['strobe'],
    codec=config['codec']
    )

Using libx264 to encode video, ignoring codec argument...
Library version: 1.8.0
Number of cameras detected: 1
Serial:  22297042
Done


In [10]:
device.cam

<pypylon.pylon.InstantCamera; proxy of <Swig Object of type 'Pylon::CInstantCamera *' at 0x7fde807bed20> >

In [11]:
device.start()

here
AcquisitionMode Continuous
ExposureAuto Off
ExposureMode Timed
ExposureTime 68295.0
BinningVertical 1
Height 1200
Width 1920
OffsetX 0
OffsetY 0
PixelFormat Mono8
AcquisitionFrameRate 15.0
GainAuto Off
Gain 1.0
Started cam acquisition
saving initialized: right_top


# Camera properties

In [9]:
import basler_utils as bs

In [10]:
dir(Basler)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cleanup_powerup_state',
 'compute_timestamp_offset',
 'initialize_metadata_saving_hdf5',
 'initialize_preview',
 'loop',
 'preview_worker',
 'process',
 'start',
 'stop',
 'stop_streaming',
 'update_settings',
 'write_metadata']

In [ ]:
device.cleanup_powerup_state()

In [10]:
device.options

{'AcquisitionMode': 'Continuous',
 'ExposureAuto': 'Off',
 'ExposureTime': 68295.0,
 'BinningVertical': 1,
 'Height': 1200,
 'Width': 1920,
 'OffsetX': 0,
 'OffsetY': 0,
 'PixelFormat': 'Mono8',
 'AcquisitionFrameRateAuto': False,
 'AcquisitionFrameRate': 15.0,
 'GainAuto': 'Off',
 'Gain': 1.0,
 'SharpnessAuto': 'Off'}

In [11]:
device.nodemap

<pypylon.genicam.INodeMap; proxy of <Swig Object of type 'GENAPI_NAMESPACE::INodeMap *' at 0x7fdc01fa4db0> >

In [12]:

for key, value in device.options.items():
    print(key, value)
    try:
        bs.set_value(device.nodemap, key, value)
    except Exception as e:
        print("error", e)
    

AcquisitionMode Continuous
ERROR setting: AcquisitionMode Continuous
error Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)
ExposureAuto Off
ERROR setting: ExposureAuto Off
error Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)
ExposureTime 68295.0
ERROR setting: ExposureTime 68295.0
error Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)
BinningVertical 1
ERROR setting: BinningVertical 1
error Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)
Height 1200
ERROR setting: Height 1200
error Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)
Width 1920
ERROR setting: Width 1920
error Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)
OffsetX 0
ERROR setting: OffsetX 0
error Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)
OffsetY 0
ER

In [12]:

nodename = 'AcquisitionMode'
value = 'Continuous'
node = device.nodemap.GetNode(nodename)

LogicalErrorException: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)

In [ ]:
device.nodemap

In [10]:
device.update_settings()

AcquisitionMode Continuous
ERROR setting: AcquisitionMode Continuous


Traceback (most recent call last):
  File "/home/julianarhee/Repositories/PIMAQ/basler_utils.py", line 181, in set_value
    node = nodemap.GetNode(nodename)
  File "/home/julianarhee/anaconda3/envs/pimaq/lib/python3.8/site-packages/pypylon/genicam.py", line 1357, in GetNode
    return _genicam.INodeMap_GetNode(self, Name)
_genicam.LogicalErrorException: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)


ValueError: Error: Map not allocated : LogicalErrorException thrown (file 'NodeMap.cpp', line 278)

In [10]:
cam['serial']

22513851

In [ ]:
device.start()

#### debug Basler instance 

In [8]:
max_cams=2
tlFactory = pylon.TlFactory.GetInstance()
devices = tlFactory.EnumerateDevices()
cam_list = pylon.InstantCameraArray(min(len(devices), max_cams))


In [9]:
for ix, c in enumerate(cam_list):
    c.Attach(tlFactory.CreateDevice(devices[ix]))
    this_serial = c.DeviceInfo.GetSerialNumber() #py.get_serial_number(c)
    print(this_serial)

22297042


In [11]:
testcam = c

In [13]:
testcam.Open()

In [17]:
c.TimestampLatch.Execute()


In [19]:
testcam.AcquisitionStart.Execute()

In [26]:
c.AcquisitionStop.Execute()

#### Node acccess

In [15]:
import pypylon

In [16]:
nodemap = c.GetNodeMap()


In [17]:
nodemap

<pypylon.genicam.INodeMap; proxy of <Swig Object of type 'GENAPI_NAMESPACE::INodeMap *' at 0x7f069a483450> >

In [19]:
nodename= 'AcquisitionMode' ##'AcquisitionFrameRateEnable'
value = 'cam['serial'], ' #'FrameStart'
node = nodemap.GetNode(nodename)
node, node.GetValue()

(<pypylon.genicam.IEnumeration; proxy of <Swig Object of type 'GENAPI_NAMESPACE::IEnumeration *' at 0x7f069a4839f0> >,
 'Continuous')

In [20]:
def get_nodeval_and_type(node):
    
    return node, type(node)
    

In [21]:
nodeval, typestring = get_nodeval_and_type(node)
nodeval, typestring


(<pypylon.genicam.IEnumeration; proxy of <Swig Object of type 'GENAPI_NAMESPACE::IEnumeration *' at 0x7f069a4839f0> >,
 pypylon.genicam.IEnumeration)

In [22]:
nodeval.GetMax()

AttributeError: 'IEnumeration' object has no attribute 'GetMax'

In [23]:
entrylist = nodeval.GetEntries()
for entry in entrylist:
   print(entry.Node.GetName())

EnumEntry_AcquisitionMode_SingleFrame
EnumEntry_AcquisitionMode_Continuous


In [24]:
typestring

pypylon.genicam.IEnumeration

#### New function for setting values

In [25]:
if typestring in [pypylon.genicam.IFloat, pypylon.genicam.IInteger]:
    assert(value <= nodeval.GetMax() and value >= nodeval.GetMin())
    if typestring == pypylon.genicam.IInteger:
        assert(type(value)==int)
        if pypylon.genicam.IsAvailable(nodeval) and pypylon.genicam.IsWritable(nodeval):
            nodeval.SetValue(value)
        else:
            raise ValueError('Node not writable or available: %s' %nodename)
    elif typestring == pypylon.genicam.IFloat:
        assert(type(value) in [float, int])
        if pypylon.genicam.IsAvailable(nodeval) and pypylon.genicam.IsWritable(nodeval):
            nodeval.SetValue(float(value))
        else:
            raise ValueError('Node not writable or available: %s' %nodename)
elif typestring == pypylon.genicam.IEnumeration:
    assert(type(value)==str)
    entry = nodeval.GetEntryByName(value)
    if entry is None:
       print('Valid entries: ')
       entrylist = nodeval.GetEntries()
       for entry in entrylist:
           print(entry.GetName())
       raise ValueError('Invalid entry!: %s' %value)
    #else:
       #entry = PySpin.CEnumEntryPtr(entry)
    if pypylon.genicam.IsAvailable(entry) and pypylon.genicam.IsReadable(entry):
       nodeval.SetIntValue(entry.GetValue())
    else:
       raise ValueError('Entry not readable!')
elif typestring == pypylon.genicam.IBoolean:
   assert(type(value)==bool)
   if pypylon.genicam.IsAvailable(nodeval) and pypylon.genicam.IsWritable(nodeval):
       nodeval.SetValue(value)
   else:
       raise ValueError('Node not writable or available: %s' %nodename)
        

In [26]:
type(value)


str

In [27]:
nodeval.GetValue()

'Continuous'

##### testing enum

In [28]:
entry = nodeval.GetEntryByName('Continuous')

In [29]:

if entry is None:
   print('Valid entries: ')
   entrylist = nodeval.GetEntries()
   for entry in entrylist:
       print(entry.GetName())
   raise ValueError('Invalid entry!: %s' %value)
#else:
   #entry = PySpin.CEnumEntryPtr(entry)
if pypylon.genicam.IsAvailable(entry) and pypylon.genicam.IsReadable(entry):
   nodeval.SetIntValue(entry.GetValue())
else:
   raise ValueError('Entry not readable!')


In [30]:
node.GetValue()

'Continuous'

In [77]:
type(node)


pypylon.genicam.IInteger

In [30]:
pypylon.genicam.IInteger == node.Node.GetPrincipalInterfaceType()

False

In [116]:
import pypylon
import importlib

In [117]:
pypylon.genicam.AccessException()

_genicam.AccessException()

In [118]:
import basler_utils as bs

In [119]:
#{'strobe': {'line': 2, 'duration': 0.0}}
line=4


In [161]:
importlib.reload(bs)

<module 'basler_utils' from '/home/julianarhee/Repositories/PIMAQ/basler_utils.py'>

In [162]:
assert(type(line)==int)
linestr = 'Line%d'%line
linestr

'Line4'

In [163]:
bs.turn_strobe_on(nodemap, line)